In [1]:
import os

In [64]:
path = '/home/tchau/code/condor_submit/OscNext/'
if not (os.path.exists(path)): os.makedirs(path)
cfig_file = '/home/tchau/code/condor_submit/OscNext/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/OscNext/Sample.submit'
with open(sub_file, 'w') as s:
    # s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Samples/OscNext_hdf5topkl.py'+'\n')
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Samples/OscNext_i3topkl.py'+'\n')

    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 8GB'+'\n')
    s.write('\n')
    s.write('arguments = -n $(nu) -s $(set)'+'\n')
    s.write('queue')
    s.close()



In [3]:
nutypes=["12", "14", "16"]
# baseline
sets=['1122']
# systs
# sets=['1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133']
submit = '/home/tchau/code/condor_submit/OscNext/submit.sh'

with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for set in sets:
        for nu in nutypes:
            path = '/home/tchau/code/condor_submit/OscNext'
            dag_file = '{}/OscNextSample_{}{}.dag'.format(path, nu, set)
            with open(dag_file, 'w') as f:
                f.write('JOB OscNextSample_{}{}'.format(nu, set)+' Sample.submit \n')
                f.write('VARS OscNextSample_{}{}'.format(nu, set)+' JOBNAME="OscNextSample_{}{}"'.format(nu, set) +' nu="{}" set="{}"'.format(nu, set)+'\n')
            f.close()
            s.write("condor_submit_dag -config dagman.config {} \n".format(dag_file))
s.close()

# Split mode

In [16]:
import glob
# Get list of file:
sample = '161122'
filenamelist = glob.glob('/data/ana/LE/oscNext/pass2/genie/level7_v02.00/{0}/oscNext_genie_level7_v02.00_pass2.{0}.*.i3.zst'.format(sample))


In [18]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
list_split = list(split(filenamelist, 10))
i=0
for listfile in list_split:
    file = f'/home/tchau/code/condor_submit/OscNext/filelist/oscNext_genie_level7_v02.00_pass2.{sample}.list{i}.txt'
    with open(file, 'w') as f:
        for l in listfile:
            f.write(l+'\n')
    i+=1

In [14]:
sub_file = '/home/tchau/code/condor_submit/OscNext/Sample_split.submit'
with open(sub_file, 'w') as s:
    # s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Samples/OscNext_hdf5topkl.py'+'\n')
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Samples/OscNext_i3topkl.py'+'\n')

    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 2GB'+'\n')
    s.write('\n')
    s.write('arguments = --listf $(listf)'+'\n')
    s.write('queue')
    s.close()


In [20]:

submit = '/home/tchau/code/condor_submit/OscNext/submit.sh'
filelist = glob.glob(f'/home/tchau/code/condor_submit/OscNext/filelist/*{sample}*.txt')

i=0
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for file in filelist:
        path = '/home/tchau/code/condor_submit/OscNext'
        dag_file = '{}/OscNextSample{}_list{}.dag'.format(path, sample, i)
        with open(dag_file, 'w') as f:
            f.write('JOB OscNextSample{}_list{}'.format(sample, i)+' Sample_split.submit \n')
            f.write('VARS OscNextSample{}_list{}'.format(sample, i)+' JOBNAME="OscNextSample{}_list{}"'.format(sample, i) +' listf="{}"'.format(file)+'\n')
        f.close()
        s.write("condor_submit_dag -config dagman.config {} \n".format(dag_file))
        i+=1
s.close()